# <center>** SVM - Support Vector Machine ** </center>

notes for running the scripts: <br>
python >= 3.8 <br>
numpy==1.18.0 <br>
pandas==1.2.3 <br>
scikit-learn==0.24.1

In [ ]:
from sklearn import datasets, svm as sk_svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# for Kernel Trick
def linear_kernel(x1, x2):
    return np.dot(x1, x2)


# for Kernel Trick
def polynomial_kernel(x1, x2, p=3):
    return (1 + np.dot(x1, x2)) ** p

In [ ]:
class SVM:
    image_counter = 0  # used to save plots

    def __init__(self, kernel=linear_kernel, C=1.0, tolerance=0.001, max_passes=10, random_state=None,
                 visualization=False, plot_plane=True):
        self.kernel = kernel  # the default uses no kernel (linear kernel)
        self.C = C  # allows margin (C=1) to margins are heavily fined (C=100)
        self.tolerance = tolerance  # tolernace for convergence
        self.max_passes = max_passes  # also a tolernace for convergence
        self.alphas = None  # aka lambdas - as stated in the dual problem
        self.kernel_results = None
        self.random_state = random_state
        self.visualization = visualization  # flag for plotting
        if self.visualization:
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(1, 1, 1)

        self.plot_plane = plot_plane

    def calc_kernel(self, X):
        self.kernel_results = np.zeros((X.shape[0], X.shape[0]))

        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                self.kernel_results[i, j] = self.kernel(X[i], X[j])

    def calc_f(self, x, b):
        f = 0.0
        for i in range(self.X.shape[0]):
            f += self.alphas[i] * self.y[i] * self.kernel(self.X[i, :], x)
        f += b
        return f

    @staticmethod
    # as part of the smo algorithm - alphaj(lambda_j) is selected randomly
    def choose_j_randomly(i, n_samples, random_state):
        j = i
        while j == i:
            j = random_state.randint(0, n_samples - 1)
        return j

    # calc higher and lower bounds on alphaj
    def calc_L_H(self, yi, yj, alphai, alphaj):
        if yi == yj:
            L = max(0.0, alphaj + alphai - self.C)
            H = min(self.C, alphaj + alphai)
        else:
            L = max(0.0, alphaj - alphai)
            H = min(self.C, self.C + alphaj - alphai)

        return L, H

    def calc_eta(self, xi, xj):
        return 2 * self.kernel(xi, xj) - self.kernel(xi, xi) - self.kernel(xj, xj)

    # separating hyper plane calculation
    def calc_w_star(self):
        return sum([self.alphas[i] * self.y[i] * self.X[i, :] for i in range(self.X.shape[0])])

    # fitting is done according to SMO
    def fit(self, X, y):
        self.X = X
        self.y = y

        n_samples, n_features = X.shape
        self.alphas = np.zeros(n_samples)
        b = 0.0
        passes = 0
        it = 0

        while passes <= self.max_passes:
            num_changed_alphas = 0
            for i in range(n_samples):
                f = self.calc_f(X[i, :], b)
                Ei = f - y[i]
                if (y[i] * Ei < -self.tolerance and self.alphas[i] < self.C) or (
                        y[i] * Ei > self.tolerance and self.alphas[i] > 0.0):
                    j = SVM.choose_j_randomly(i, n_samples, self.random_state)
                    f = self.calc_f(X[j, :], b)
                    Ej = f - y[j]
                    alpha_i_old, alpha_j_old = self.alphas[i], self.alphas[j]
                    L, H = self.calc_L_H(y[i], y[j], self.alphas[i], self.alphas[j])
                    if abs(L - H) < 1e-4:
                        continue
                    eta = self.calc_eta(X[i, :], X[j, :])
                    if eta >= 0:
                        continue

                    # calc alpha_j
                    self.alphas[j] -= y[j] * (Ei - Ej) / eta
                    if self.alphas[j] > H:
                        self.alphas[j] = H
                    elif self.alphas[j] < L:
                        self.alphas[j] = L

                    if abs(self.alphas[j] - alpha_j_old) < 1e-4:
                        continue

                    self.alphas[i] += y[i] * y[j] * (alpha_j_old - self.alphas[j])
                    b1 = b - Ei - y[i] * (self.alphas[i] - alpha_i_old) * self.kernel(X[i, :], X[i, :]) - y[j] * (
                            self.alphas[j] - alpha_j_old) * self.kernel(X[i, :], X[j, :])
                    b2 = b - Ej - y[i] * (self.alphas[i] - alpha_i_old) * self.kernel(X[i, :], X[j, :]) - y[j] * (
                            self.alphas[j] - alpha_j_old) * self.kernel(X[j, :], X[j, :])

                    if 0 < self.alphas[i] < self.C:
                        b = b1
                    elif 0 < self.alphas[j] < self.C:
                        b = b2
                    else:
                        b = (b1 + b2) / 2

                    num_changed_alphas += 1
            print(f'{it=}:{self.alphas=}')
            it += 1

            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0

            if self.visualization:
                self.w_star = self.calc_w_star()
                self.b_star = b
                self.visualize()

        self.w_star = self.calc_w_star()
        self.b_star = b

        if self.visualization:
            self.visualize(show=True)

    # label prediction
    def predict(self, x):
        return np.sign(self.b_star + sum(
            [self.alphas[i] * self.y[i] * self.kernel(self.X[i, :], x) for i in range(self.X.shape[0])]))

    # plotting
    def plot_2d_data(self):
        target_plus_indicies = np.where(self.y == 1)[0]
        target_minus_indicies = np.where(self.y == -1)[0]
        sv_indicies = np.where(self.alphas > 0)[0]
        self.ax.plot(self.X[target_plus_indicies, 0], self.X[target_plus_indicies, 1], 'o', label=1)
        self.ax.plot(self.X[target_minus_indicies, 0], self.X[target_minus_indicies, 1], 'o', label=-1)
        self.ax.plot(self.X[sv_indicies, 0], self.X[sv_indicies, 1], 'o', markersize=14, markerfacecolor="None",
                     label='sv')
        plt.legend()
        self.ax.set_xlabel('x1')
        self.ax.set_ylabel('x2')
        self.ax.axis('equal')
        # plt.show()

    # plotting
    def visualize(self, show=False):
        self.ax.cla()

        self.plot_2d_data()
        min_feature_value = np.min(self.X)
        max_feature_value = np.max(self.X)
        self.ax.set_xlim(min_feature_value, max_feature_value)
        self.ax.set_ylim(min_feature_value, max_feature_value)

        # hyperplane = x.w+b
        # v = x.w+b
        # psv = 1
        # nsv = -1
        # dec = 0
        def hyperplane(x, w, b, v):
            return (-w[0] * x - b + v) / w[1]

        datarange = (min_feature_value * 0.9, max_feature_value * 1.1)
        hyp_x_min = datarange[0]
        hyp_x_max = datarange[1]

        if self.plot_plane:
            # (w.x+b) = 1
            # positive support vector hyperplane
            psv1 = hyperplane(hyp_x_min, self.w_star, self.b_star, 1)
            psv2 = hyperplane(hyp_x_max, self.w_star, self.b_star, 1)
            self.ax.plot([hyp_x_min, hyp_x_max], [psv1, psv2], 'k')

            # (w.x+b) = -1
            # negative support vector hyperplane
            nsv1 = hyperplane(hyp_x_min, self.w_star, self.b_star, -1)
            nsv2 = hyperplane(hyp_x_max, self.w_star, self.b_star, -1)
            self.ax.plot([hyp_x_min, hyp_x_max], [nsv1, nsv2], 'k')

            # (w.x+b) = 0
            # positive support vector hyperplane
            db1 = hyperplane(hyp_x_min, self.w_star, self.b_star, 0)
            db2 = hyperplane(hyp_x_max, self.w_star, self.b_star, 0)
            self.ax.plot([hyp_x_min, hyp_x_max], [db1, db2], 'y--')

        self.plot_contours(cmap=plt.cm.coolwarm, alpha=0.8)

        precounter = ''
        for i in range(3 - len(str(SVM.image_counter))):
            precounter += '0'

        plt.title('iteration ' + str(SVM.image_counter))
        if not show:
            plt.savefig('image_' + precounter + str(SVM.image_counter) + '.png')
            SVM.image_counter += 1
            plt.pause(0.01)
        else:
            plt.savefig('image_' + precounter + str(SVM.image_counter) + '.png')
            SVM.image_counter += 1
            plt.show()

    # plotting
    def make_meshgrid(self, h=0.02):
        x_min, x_max = np.min(self.X[:, 0]) - 1.5, np.max(self.X[:, 0]) + 1.5
        y_min, y_max = np.min(self.X[:, 1]) - 1.5, np.max(self.X[:, 1]) + 1.5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        return xx, yy

    # plotting
    def plot_contours(self, **params):
        xx, yy = self.make_meshgrid()
        Z = np.array([self.predict([xx_i, yy_i]) for xx_i, yy_i in zip(xx, yy)])
        # Z = self.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        out = self.ax.contourf(xx, yy, Z, **params)
        return out

the model: find best seperating hyperplane seperator between 2 classes.<br>
best in the sense of maximizing the minimal hyperplane seperator distance.


<center><img src="images/svm_general_4.png"></center>

The seperating hyperplane can be written as the set of points $\vec x$ satisfiying:
$$(1)\; \vec w^T \cdot \vec x  +b = 0$$

# mathematical formulation:
$$(2)\;f(\vec x,\vec w,b)=sign(\vec w^T \cdot \vec x +b)$$
$$(3)\;Margin( \vec w ,b) = min Margin(x_i)$$
$$(4)\;(\vec w,b) = argmax Margin(\vec w ,b)$$

<center><img src="images/svm_2.png"></center>

Let's define $$(5)\;n=\dfrac {\vec w} {||\vec w||}$$ $$(6)\;\vec w^T \cdot \vec {x^+}  +b = 1$$ $$(7)\;\vec w^T \cdot \vec {x^-}  +b = -1$$
(6) and (7) are called the suppurt vectors. <br>
note the the decision boundary remains: $\vec w^T \cdot \vec x  +b = 0$. <br> 
so given a new vector $\vec x_i$ the class will be determined as stated in (2): $sign(\vec w^T \cdot \vec x_i +b)$

so the margin we want to maximize is:
$$(8)\; Margin = (x^+ - x^-) \cdot n = (x^+ - x^-) \cdot \dfrac {\vec w} {||\vec w||}$$
applying (5) to (8):
$$(9)\;  Margin = (x^+ - x^-) \cdot \dfrac {\vec w} {||\vec w||}$$
applying (6) and (7) to (8):
$$(10)\; Margin = (\dfrac {1-b} {\vec w} - \dfrac {-1-b} {\vec w})\cdot \dfrac {\vec w} {||\vec w||} =\dfrac 2 {||\vec w||}$$

Maximizing $\dfrac 2 {||w||}$ can be expressed as Minimizing: $$(11)\;\dfrac 1 2 ||\vec w||^2$$

Let's define $y_i \in \{1,-1\}$ to be the class label of $\vec x_i$, $i=\{1,...,n\}$. <br>
note that the constraints (6) and (7) can be written as:
$$(12)\;y_i(\vec w^T \cdot \vec {x}  +b)-1 \geq 0$$


**finalizing the formulation:**<br>
Minimizing: $$\dfrac 1 2 ||\vec w||^2$$
subject to:
$$y_i(\vec w^T \cdot \vec {x}  +b)-1 \geq 0$$
The above formulation is an optimization problem with a convex quadratic objective and linear constraints, it can be solved using QP solvers.

## Hard Margin

the above formulation does not allow errors.<br>
let's define a hyperplane and randomly draw point.<br>


In [ ]:
def example_1():
    # EXAMPLE 1
    random_state = np.random.RandomState(0)
    X = random_state.randn(10, 2)
    b = -0.2
    w = np.array([0.5, -0.3])
    y = np.sign(b + np.dot(X, w))

    svm = SVM(C=100.0, random_state=random_state, visualization=False)
    svm.fit(X, y)

example_1()

<center><img src="images/example_1_c100_new_2.gif"></center>

the final result:
<center><img src="images/example_1_C_100_final_new.png"></center>

# Lagrange Duality

The general Lagrangian function:
$$ (13)\;\mathcal{L}(x,\lambda,\nu) = f_0(x)+ \sum_{i=1}^{m} \lambda_i f_i(x)+\sum_{i=1}^{p} \nu_i h_i()$$
applying it to our case, where:
$$(14)\;f_0(w) = \dfrac 1 2 ||\vec w||^2$$
$$(15)\;f_1(w,b) = -y_i(\vec w^T \cdot \vec {x_i}  +b)+1 \leq 0$$
$$(16)\; \mathcal{L}(w,b,\lambda) = \dfrac 1 2 ||\vec w||^2-\sum_{i=1}^{m}\lambda_i \{y_i(\vec w^T \cdot \vec {x_i}  +b)+1\}$$

we'll use the KKT conditions:
$$(17)\; \dfrac{\partial{}}{\partial w} \mathcal{L}(w,b,\lambda)=w-\sum_{i=1}^{m}\lambda_i y_i x_i = 0$$
so:
$$(18)\;w=\sum_{i=1}^{m}\lambda_i y_i x_i $$
$$(19)\; \dfrac{\partial{}}{\partial b} \mathcal{L}(w,b,\lambda)=\sum_{i=1}^{m}\lambda_i y_i = 0$$

from (13) and (18):

$$(20)\; \mathcal{L}(w,b,\lambda)= \sum_{i=1}^{m}\lambda_i - \dfrac 1 2 \sum_{i,j=1}^{m} y_i y_j \lambda_i \lambda_j \vec x_i ^ T \vec x_j - b\sum_{i,j=1}^{m} \lambda_i y_i$$

adding (19) to (20):
$$(21)\; \mathcal{L}(w,b,\lambda)= \sum_{i=1}^{m}\lambda_i - \dfrac 1 2 \sum_{i,j=1}^{m} y_i y_j \lambda_i \lambda_j \vec x_i ^ T \vec x_j$$

the dual optimization problem can be expressed as Maximizing:
 $$  \mathcal{L}(\lambda)= \sum_{i=1}^{m}\lambda_i - \dfrac 1 2 \sum_{i,j=1}^{m} y_i y_j \lambda_i \lambda_j \langle\vec x_i, \vec x_j\rangle  $$
 s.t.
 $$ \lambda_i \geq 0, i=1,...,m$$
 $$\sum_{i=1}^{m}\lambda_i y_i = 0$$

note that after finding the $\lambda$s we need to go back to the primal problem and find $w^*$ and $b^*$. <br>
$w^*$ can be extracted directly from (18).
$$(22)\; b^* = -\dfrac 1 2 \{max_{i:y_i=-1} w^{*^T} x_i + min_{i:y_i=1} w^{*^T}x_i \} $$
also note that now (2) can be expressed as:
$$ (23)\;f(\vec x,\vec w,b)=sign(\vec w^T \cdot \vec x +b) = sign((\sum_{i=1}^{m}\lambda_i y_i x_i)^T\vec x +b)=sign(\sum_{i=1}^{m}\lambda_i y_i \langle \vec x_i, \vec x \rangle +b)$$

let's examine it using the well-known iris dataset (taking into account only 2 labels and 2 features).

In [ ]:
def example_2():
    ## EXAMPLE 2
    iris = datasets.load_iris()
    X = iris['data'][:, (2, 3)]
    scaler = StandardScaler()
    Xstan = scaler.fit_transform(X)

    data = pd.DataFrame(data=Xstan, columns=['petal length', 'petal width'])
    data['target'] = iris['target']
    data = data[data['target'] != 2]  # we will only focus on Iris-setosa and Iris-Versicolor
    data['target'].iloc[data['target'] == 0] = -1

    random_state = np.random.RandomState(0)
    svm = SVM(C=100.0, random_state=random_state, visualization=False)
    svm.fit(data.loc[:, ['petal length', 'petal width']].values, data['target'].values)

example_2()

<center><img src="images/example_2_C_100_new.gif"></center>

the final result:
<center><img src="images/example_2_C_100__final_new.png"></center>

# Soft Margin

what will we do when we have inconsistent data (some outlier)?
Let's take the first example, and libertly flip a label

In [ ]:
def example_1_1():
    # EXAMPLE 1_1
    random_state = np.random.RandomState(0)
    X = random_state.randn(10, 2)
    b = -0.2
    w = np.array([0.5, -0.3])
    y = np.sign(b + np.dot(X, w))
    y[3] = -y[3]

    svm = SVM(C=100.0, random_state=random_state, visualization=False)
    svm.fit(X, y)

example_1_1()

<center><img src="images/example_1_1_C_100_new.gif"></center>

the solution: allow slacks 
<br>

we can change (11) to satisfy our new need:<br> 
Minimizing: $$(24)\;\dfrac 1 2 ||\vec w||^2 + C \sum_{i=1}^{m} \xi_i$$
subject to:
$$(25)\;y_i(\vec w^T \cdot \vec {x}  +b)\geq 1-\xi_i$$
$$(26)\;\xi_i \geq 0$$
<br>
$C$ controls the relative weighting between the twin goals of making ||\vec w||^2 small and of ensuring that most examples have functional margin at least 1.

<br>
reforming the lagrangian yields:

$$(27)\; \mathcal{L}(w,b,\lambda) = \dfrac 1 2 w^T w+C\sum_{i=1}^{m}\xi_i-\sum_{i=1}^{m}\lambda_i \{y_i(\vec w^T \cdot \vec {x_i}  +b)-1+\xi_i\}-\sum_{i=1}^{m}r_i \xi_i$$

<br>

$\lambda_i$ and $r_i$ are the lagrangian multipliers.

<br>
after derivation we get:

$$(28)\; \mathcal{L}(w,b,\lambda)= \sum_{i=1}^{m}\lambda_i - \dfrac 1 2 \sum_{i,j=1}^{m} y_i y_j \lambda_i \lambda_j \vec x_i ^ T \vec x_j$$

s.t.
 $$(29)\;  0 \leq \lambda_i \leq C, i=1,...,m$$
 $$(30)\; \sum_{i=1}^{m}\lambda_i y_i = 0$$
 <br>
 
 $w^*$ can again be extracted directly from (18).
 <br>

 note that KKT conditions:
 $$(31)\;  \lambda_i=0: y_i(\vec w^T \cdot \vec {x_i}  +b) \geq 1$$
 $$(32)\;  \lambda_i=C: y_i(\vec w^T \cdot \vec {x_i}  +b) \leq 1$$
 $$(33)\;  0<\lambda_i<C: y_i(\vec w^T \cdot \vec {x_i}  +b) = 1$$

Let's view the Iris example again with a changed label:

In [ ]:
def example_2_1():
    ## EXAMPLE 2_1
    iris = datasets.load_iris()
    X = iris['data'][:, (2, 3)]
    scaler = StandardScaler()
    Xstan = scaler.fit_transform(X)

    data = pd.DataFrame(data=Xstan, columns=['petal length', 'petal width'])
    data['target'] = iris['target']
    data = data[data['target'] != 2]  # we will only focus on Iris-setosa and Iris-Versicolor
    data['target'].iloc[data['target'] == 0] = -1
    # data = data.loc[(0,1,2,3,4,90,91,92,93,94),:]
    data['target'].iloc[20] = -data['target'].iloc[20]

    random_state = np.random.RandomState(0)
    svm = SVM(C=100.0, random_state=random_state, visualization=False)
    svm.fit(data.loc[:, ['petal length', 'petal width']].values, data['target'].values)

example_2_1()

<center><img src="images/example_2_1_C_100_new.gif"></center>

now, changing to C=1.0:

In [ ]:
def example_2_2():
    ## EXAMPLE 2_2
    iris = datasets.load_iris()
    X = iris['data'][:, (2, 3)]
    scaler = StandardScaler()
    Xstan = scaler.fit_transform(X)

    data = pd.DataFrame(data=Xstan, columns=['petal length', 'petal width'])
    data['target'] = iris['target']
    data = data[data['target'] != 2]  # we will only focus on Iris-setosa and Iris-Versicolor
    data['target'].iloc[data['target'] == 0] = -1
    # data = data.loc[(0,1,2,3,4,90,91,92,93,94),:]
    data['target'].iloc[20] = -data['target'].iloc[20]

    random_state = np.random.RandomState(0)
    svm = SVM(C=1.0, random_state=random_state, visualization=False)
    svm.fit(data.loc[:, ['petal length', 'petal width']].values, data['target'].values)

example_2_2()

<center><img src="images/example_2_1_C_1_new.gif"></center>

the SVM yields the wanted seperating plane despite the error.

# SMO - Sequential Minimal Optimization


after getting to the lagrangian formulation, we need to find the corresponding lagrange multipliers in order to get w, b in the primal problem.<br>

Let's assume that we have a set of $\lambda_i$ that satisfy (29) and (30).
what if we could change one $\lambda_i$ in order to increase $\mathcal{L}$ but from (30):
$$\lambda_1 y_1 = -\sum_{i=2}^{m}\lambda_i y_i$$
by multiplying by $y_i$:
$$\lambda_1 = -y_1 \sum_{i=2}^{m}\lambda_i y_i$$

it's not possible since one change affects the other.

the solution: <br>
update 2 $\lambda_i$s simultaneously in order to keep the constarints.

while not converge{ <br>
    &emsp; select $\lambda_i$, $\lambda_j$ (heuristicly towards global maximum) <br>
    &emsp; optimize $\mathcal{L}$ with corresponding $\lambda_i$, $\lambda_j$ (other $\lambda$ remain fixed) <br>
} <br>

testing convergence: <br>
check that (31),(32),(33) are within tolerance bounds.

Let's assume that we have a set of $\lambda_i$ that satisfies (29),(30)and we chose  $\lambda_1$, $\lambda_2$ to optimize.<br>
from (30): <br>
$$\lambda_1 y_1 + \lambda_2 y_2 = -\sum_{i=3}^{m}\lambda_i y_i$$
the right-hand side is fixed, so:
$$(34)\; \lambda_1 y_1 + \lambda_2 y_2 = \zeta = const$$
from (29) we know that $\lambda_1$, $\lambda_2$ lie within $[ 0,C ]$.


<center><img src="images/L_H_lambda.png"></center>


so we have a lower boud L and upper bound H for $\lambda_2$: <br>

$$ L \leq \lambda_2 \leq H $$

we can rewrite (34) as:
$$ (35)\; \lambda_1  = (\zeta -\lambda_2 y_2) y_1 $$

and now also:
$$\mathcal{L}(\lambda_1,\lambda_2,...\lambda_m) = \mathcal{L}((\zeta -\lambda_2 y_2) y_1,\lambda_2,...\lambda_m)$$

remember that $\lambda_3,...\lambda_m$ are consts. <br>
we get a quadratic function in $\lambda_2$ that can be expressed for some $a,b,c$:
$$a\lambda_2^2+b\lambda_2+c$$
to maximize the above we can derive and compare to 0, remembering we are bounded by L and H so:
$$  \lambda_2^{new} > H : \lambda_2^{new} = H$$
$$  \lambda_2^{new} < H : \lambda_2^{new} = L$$
$$  L<\lambda_2^{new} < H: \lambda_2^{new}=\lambda_2^{new}$$

$\lambda_1$ can be extracted from (35).


as for proof of convergence (credit to Yoni):<br> it is based on a theorem that shows the ability to decompose the QP into smaller QP problems, solutions of which guarantee the solution of the original QP. <br> reference to the theorem: <br> https://ieeexplore.ieee.org/document/622408


full SMO article:<br>
https://www.microsoft.com/en-us/research/publication/sequential-minimal-optimization-a-fast-algorithm-for-training-support-vector-machines/

# Kernel

what about linearity? <br>
in a simple example the SVM cannot find a linear seperator. <br>
let's see the following example:

In [ ]:
def example_3():
    ## EXAMPLE 3
    random_state = np.random.RandomState(0)
    X, y = datasets.make_moons(noise=0.1, random_state=2)
    y[y == 0] = -1
    svm = SVM(C=1.0, random_state=random_state, visualization=True)
    svm.fit(X, y)

example_3()

<center><img src="images/example_3_new.gif"></center>

no linear seperator - can converge (after a while) but is it what we're looking for?
<center><img src="images/example_3_new_final.png"></center>

Kernel to the rescue.<br>
we can manipulate the data to a higher dimension, eventually - a linear seperator will exist.<br>
we won't go in to the details, but the "Kernel Trick" allows us to "enjoy" the higher dimension with lower dimension calculations.<br>
further details:<br>
https://towardsdatascience.com/the-kernel-trick-c98cdbcaeb3f

we'll apply a polynomial kernel to the above example:

In [ ]:
def example_4():
    ## EXAMPLE 4
    random_state = np.random.RandomState(0)
    X, y = datasets.make_moons(noise=0.1, random_state=2)
    y[y == 0] = -1
    svm = SVM(C=1.0, kernel=gaussian_kernel, random_state=random_state, visualization=True, plot_plane=False)
    svm.fit(X, y)

example_4()

<center><img src="images/example_4_new.gif"></center>

note that the colors represent the different classes, since we are not talking about 2d space.

# Real DataSets

our first try was with facial expression recognition.<br>
for further details, check:<br>
https://www.kaggle.com/ashishpatel26/facial-expression-recognitionferchallenge

<center><img src="images/fer.png"></center>

The training set consists of 28,709 examples, with 48*48 pixels.<br>
it appears that we were not up to the challnge, we only got around 57% accuracy. <br>
(accuracy=0.57, precision=1.0, recall=0.0077, f1=0.01538)
<br> <br>
different things we tried to do:
<br>
1. slicing the examples <br>
1. taking only half-picture <br>

In [ ]:
# download fer2013.csv form the link above and rename it to train.csv
def example_5():
    ## EXAMPLE 5
    df = pd.read_csv('train.csv')
    df1 = df[np.logical_or(df['emotion'] == 3, df['emotion'] == 4)]
    img_array = df1.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48*48).astype('float32'))


    X = np.stack(img_array, axis=0)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # consider only the bottom half
    # X = X[:, range(int(48*48/2), X.shape[1])]
    # X = X[:, range(int(48 * 48 / 2))]
    
    y = df1['emotion'].values

    y[y == 3] = -1
    y[y == 4] = 1

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # X = img_array[range(100),:]
    # y = df1['emotion'].values[range(100)]
    X_train = X_train[range(500), :]
    y_train = y_train[range(500)]
    X_test = X_test[range(300), :]
    y_test = y_test[range(300)]
    random_state = np.random.RandomState(0)
    svm = SVM(C=1.0, kernel=gaussian_kernel, random_state=random_state, visualization=False)
    svm.fit(X_train, y_train)

    # accuracy = 0
    # for x, actual_y in zip(X_test, y_test):
    #     pred_y = svm.predict(x)
    #     if pred_y == actual_y:
    #         accuracy += 1

    # accuracy /= X_test.shape[0]
    # print(f'{accuracy=}')
    accuracy = 0
    false_positive = 0
    false_negative = 0
    true_positive = 0
    for x, actual_y in zip(X_test, y_test):
        pred_y = svm.predict(x)
        if pred_y == actual_y:
            accuracy += 1
            if pred_y == 1:
                true_positive += 1
        else:
            if pred_y == 1:
                false_positive += 1
            else:
                false_negative += 1

    print('OUR SVM, using SMO:')
    accuracy /= X_test.shape[0]
    print(f'{accuracy=}')
    precision = true_positive / (true_positive + false_positive)
    print(f'{precision=}')
    recall = true_positive / (true_positive + false_negative)
    print(f'{recall=}')
    f1 = 2 * precision * recall / (precision + recall)
    print(f'{f1=}')
    print('')

example_5()

so we went after another dataset, this time breast cancer.<br>
link:<br>
https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
<br><br>
the taget is to calssify if a tummor is malignant or benign. <br>
the data contains 30 featues and 569 instances split into 381 train instances and 188 test instances.

In [ ]:
def example_6():
    ## EXAMPLE 6
    # download data.csv form the link above
    df = pd.read_csv('data.csv')
    X = df[['radius_mean', 'texture_mean', 'perimeter_mean',
           'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
           'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
           'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
           'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
           'fractal_dimension_se', 'radius_worst', 'texture_worst',
           'perimeter_worst', 'area_worst', 'smoothness_worst',
           'compactness_worst', 'concavity_worst', 'concave points_worst',
           'symmetry_worst', 'fractal_dimension_worst']].values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    y = df['diagnosis'].values
    y[y=='M'] = -1
    y[y=='B'] = 1

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    random_state = np.random.RandomState(0)
    svm = SVM(C=1.0, kernel=polynomial_kernel, random_state=random_state, visualization=False)
    svm.fit(X_train, y_train)

    accuracy = 0
    false_positive = 0
    false_negative = 0
    true_positive = 0
    for x, actual_y in zip(X_test, y_test):
        pred_y = svm.predict(x)
        if pred_y == actual_y:
            accuracy += 1
            if pred_y == 1:
                true_positive += 1
        else:
            if pred_y == 1:
                false_positive += 1
            else:
                false_negative += 1

    print('OUR SVM, using SMO:')
    accuracy /= X_test.shape[0]
    print(f'{accuracy=}')
    precision = true_positive / (true_positive + false_positive)
    print(f'{precision=}')
    recall = true_positive / (true_positive + false_negative)
    print(f'{recall=}')
    f1 = 2 * precision * recall / (precision + recall)
    print(f'{f1=}')
    print('')

    y_train = y_train.astype('int')
    clf = sk_svm.SVC(C=1.0, kernel='poly', random_state=42).fit(X_train, y_train)
    accuracy = 0
    false_positive = 0
    false_negative = 0
    true_positive = 0
    for x, actual_y in zip(X_test, y_test):
        pred_y = clf.predict(x.reshape(1, -1))
        if pred_y == actual_y:
            accuracy += 1
            if pred_y == 1:
                true_positive += 1
        else:
            if pred_y == 1:
                false_positive += 1
            else:
                false_negative += 1

    print('sklearn:')
    accuracy /= X_test.shape[0]
    print(f'{accuracy=}')
    precision = true_positive / (true_positive + false_positive)
    print(f'{precision=}')
    recall = true_positive / (true_positive + false_negative)
    print(f'{recall=}')
    f1 = 2 * precision * recall / (precision + recall)
    print(f'{f1=}')


OUR SVM, using SMO: <br> <br>
accuracy =  0.925531914893617 <br>
precision = 0.9421487603305785 <br>
recall =    0.9421487603305785 <br>
f1 =        0.9421487603305785 <br>
<br>
we compared the result to sklearn's SVC, using the same parameters: <br>  <br>
accuracy =  0.8882978723404256 <br>
precision = 0.852112676056338 <br>
recall =    1.0 <br>
f1 =        0.9201520912547528 <br>